In [ ]:
#| default_exp autocoder

In [ ]:
#| include: false
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from causalnlp.analyzers import ZeroShotClassifier, TextEncoder, TopicModel

class Autocoder:
    """
    Autocodes text fields
    """
    def __init__(self, verbose=1, device=None):
        """
        Instantiates the Autocoder instance.
        """
        self.v = verbose
        self.zsl = ZeroShotClassifier(device=device)
        self.device = device


    def _format_to_df(self, results, df):
        d = {}
        for e in results:
            if isinstance(e, dict): e = e.items()
            for tup in e:
                label = tup[0]
                prob = tup[1]
                lst = d.get(label, [])
                lst.append(prob)
                d[label] = lst
        new_df = df.join(pd.DataFrame(d, index=df.index))      
        return new_df
    
    def _binarize_df(self, df, colnames, threshold=0.5):
        """
        Binarizes each column in `colnames` based on threshold.
        """
        for col in colnames:
            df[col] = (df[col] >= threshold).astype(int)
        return df
 
    def _check_columns(self, labels, df):
        """check columns"""
        cols = df.columns.values
        for l in labels:
            if l in cols:
                raise ValueError('There is already a column named %s in your DataFrame.' % (l))
        

    def code_sentiment(self, docs, df, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for positive or negative sentiment
        """
        labels = ['negative', 'positive']
        self._check_columns(labels, df)

        results = self.zsl.predict(docs, labels=labels, include_labels=True, multilabel=False,
                              batch_size=batch_size,
                              nli_template="The sentiment of this movie review is {}.")
        df= self._format_to_df(results, df)   
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df
        
    def code_emotion(self, docs, df, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for emotion
        """
        labels = ['joy', 'anger', 'fear', 'sadness']
        self._check_columns(labels, df)

        results = self.zsl.predict(docs, labels=labels, include_labels=True, multilabel=False,
                              batch_size=batch_size,
                              nli_template="The emotion of this text is {}.")
        df= self._format_to_df(results, df)   
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df       
    
    def code_custom_topics(self, docs, df, labels, batch_size=8, binarize=False, threshold=0.5):
        """
        Autocodes text for user-specified topics.
        The `label` field is the name of the topic as a string (or a list of them.)
        """
        self._check_columns(labels, df)
           
        results = self.zsl.predict(docs, labels=labels, include_labels=True, batch_size=8)
        df = self._format_to_df(results, df)    
        if binarize: df = self._binarize_df(df, labels, threshold=threshold)
        return df
    
    def code_lda_topics(self, docs, df, k=10, n_features=10000):
        """
        Encode texts as semantically meaningful vectors using Latent Dirichlet Alocation
        """
        tm = TopicModel(docs, n_topics=k, n_features=n_features)
        tm.build(docs)
        e = tm.doc_topics
        topic_names = ['|'.join(t.split()[:3]) for t in tm.get_topics()]
        results = []
        for row, data in enumerate(e):
            vals = [v for v in data]
            results.append( list(zip(topic_names, vals)) )
        df = self._format_to_df(results, df)
        return df  
    
    def code_callable(self, docs, df, fn):
        """
        Autocodes text for any user-specified function
        The `fn` parameter must be a Callable and return a dictionary for each
        text in `docs` where the keys are desired column names and values are scores
        or probabilities.
        """
        
        results = []
        for doc in docs:  results.append(fn(doc))
        df = self._format_to_df(results, df)    
        return df

    def code_transformer(self, docs, df, batch_size=32, 
                         model_name='stsb-roberta-large', show_progress_bar=False):
        """
        Encode texts as semantically meaningful vectors using a Transformer model
        """
        te = TextEncoder(device=self.device, model_name=model_name)
        e = te.encode(docs, batch_size=batch_size, show_progress_bar=show_progress_bar)
        results = []
        for row, data in enumerate(e):
            keys = ["e_%04d" %(i) for i in range(data.shape[0])]
            vals = [v for v in data]
            results.append( list(zip(keys, vals)) )
        df = self._format_to_df(results, df)
        return df                       

In [ ]:
show_doc(Autocoder.code_sentiment)

---

### Autocoder.code_sentiment

>      Autocoder.code_sentiment (docs, df, batch_size=8, binarize=False,
>                                threshold=0.5)

*Autocodes text for positive or negative sentiment*

Let's prepare a toy dataset:

In [ ]:
ac = Autocoder()
reviews = ["I loved this doctor!", "This doctor was absolutely terrible."]
df = pd.DataFrame({
    'gender': ['female', 'male'],
     'review' : reviews,
      })
df.head()

,gender,review
0,female,I loved this doctor!
1,male,This doctor was absolutely terrible.


After autocoding for sentiment, the dataframe now has extra columns:

In [ ]:
result_df = ac.code_sentiment(df['review'].values, df)
result_df.head()

,gender,review,negative,positive
0,female,I loved this doctor!,0.005034,0.994966
1,male,This doctor was absolutely terrible.,0.981789,0.018211


In [ ]:
assert result_df[result_df['gender']=='female']['negative'].values[0] < 0.1
assert result_df[result_df['gender']=='female']['positive'].values[0] > 0.9
assert result_df[result_df['gender']=='male']['negative'].values[0] > 0.9
assert result_df[result_df['gender']=='male']['positive'].values[0] < 0.1

In [ ]:
show_doc(Autocoder.code_custom_topics)

---

### Autocoder.code_custom_topics

>      Autocoder.code_custom_topics (docs, df, labels, batch_size=8,
>                                    binarize=False, threshold=0.5)

*Autocodes text for user-specified topics.
The `label` field is the name of the topic as a string (or a list of them.)*

Let's prepare a toy dataset:

In [ ]:
comments = ["What is your favorite sitcom of all time?", 'I cannot wait to vote!']
df = pd.DataFrame({
    'over_18': ['yes', 'no'],
     'comments' : comments,
      })
df.head()

,over_18,comments
0,yes,What is your favorite sitcom of all time?
1,no,I cannot wait to vote!


After autocoding, the dataframe has a new column for each custom topic:

In [ ]:
result_df = ac.code_custom_topics(df['comments'].values, df, labels=['television', 'film', 'politics'])
result_df.head()

,over_18,comments,television,film,politics
0,yes,What is your favorite sitcom of all time?,0.981327,0.012260,0.000157
1,no,I cannot wait to vote!,0.000518,0.004943,0.936988


In [ ]:
assert result_df[result_df['over_18']=='yes']['television'].values[0] > 0.9
assert result_df[result_df['over_18']=='yes']['film'].values[0] < 0.1
assert result_df[result_df['over_18']=='yes']['politics'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['television'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['film'].values[0] < 0.1
assert result_df[result_df['over_18']=='no']['politics'].values[0] > 0.9

In [ ]:
show_doc(Autocoder.code_emotion)

---

### Autocoder.code_emotion

>      Autocoder.code_emotion (docs, df, batch_size=8, binarize=False,
>                              threshold=0.5)

*Autocodes text for emotion*

In [ ]:
comments = ["I'm nervous about tomorrow.", 'I got a promotion at work!',
            "My best friend was in a car accident.", "I hate it when I'm cut off in traffic."]
df = pd.DataFrame({
    'over_18': ['yes', 'no', 'yes', 'yes'],
     'comments' : comments,
      })
df.head()

,over_18,comments
0,yes,I'm nervous about tomorrow.
1,no,I got a promotion at work!
2,yes,My best friend was in a car accident.
3,yes,I hate it when I'm cut off in traffic.


In [ ]:
result_df = ac.code_emotion(df['comments'].values, df, binarize=True)
result_df.head()

,over_18,comments,joy,anger,fear,sadness
0,yes,I'm nervous about tomorrow.,0,0,1,0
1,no,I got a promotion at work!,1,0,0,0
2,yes,My best friend was in a car accident.,0,0,0,1
3,yes,I hate it when I'm cut off in traffic.,0,1,0,0


In [ ]:
assert result_df.iloc[0]['fear'] == 1
assert result_df.iloc[1]['joy'] == 1
assert result_df.iloc[2]['sadness'] == 1
assert result_df.iloc[3]['anger'] == 1

In [ ]:
show_doc(Autocoder.code_transformer)

---

### Autocoder.code_transformer

>      Autocoder.code_transformer (docs, df, batch_size=32, model_name='stsb-
>                                  roberta-large', show_progress_bar=False)

*Encode texts as semantically meaningful vectors using a Transformer model*

In [ ]:
reviews = ["I loved this doctor!", "This doctor was absolutely terrible."]
df = pd.DataFrame({
    'gender': ['female', 'male'],
     'review' : reviews,
      })
df.head()

,gender,review
0,female,I loved this doctor!
1,male,This doctor was absolutely terrible.


In [ ]:
#| notest
df = ac.code_transformer(df.review.values, df)

In [ ]:
#| notest
df.head()

,gender,review,e_0000,e_0001,e_0002,e_0003,e_0004,e_0005,e_0006,e_0007,e_0008,e_0009,e_0010,e_0011,e_0012,e_0013,e_0014,e_0015,e_0016,e_0017,e_0018,e_0019,e_0020,e_0021,e_0022,e_0023,e_0024,e_0025,e_0026,e_0027,e_0028,e_0029,e_0030,e_0031,e_0032,e_0033,e_0034,e_0035,e_0036,e_0037,e_0038,e_0039,e_0040,e_0041,e_0042,e_0043,e_0044,e_0045,e_0046,e_0047,e_0048,e_0049,e_0050,e_0051,e_0052,e_0053,e_0054,e_0055,e_0056,e_0057,e_0058,e_0059,e_0060,e_0061,e_0062,e_0063,e_0064,e_0065,e_0066,e_0067,e_0068,e_0069,e_0070,e_0071,e_0072,e_0073,e_0074,e_0075,e_0076,e_0077,e_0078,e_0079,e_0080,e_0081,e_0082,e_0083,e_0084,e_0085,e_0086,e_0087,e_0088,e_0089,e_0090,e_0091,e_0092,e_0093,e_0094,e_0095,e_0096,e_0097,e_0098,e_0099,e_0100,e_0101,e_0102,e_0103,e_0104,e_0105,e_0106,e_0107,e_0108,e_0109,e_0110,e_0111,e_0112,e_0113,e_0114,e_0115,e_0116,e_0117,e_0118,e_0119,e_0120,e_0121,e_0122,e_0123,e_0124,e_0125,e_0126,e_0127,e_0128,e_0129,e_0130,e_0131,e_0132,e_0133,e_0134,e_0135,e_0136,e_0137,e_0138,e_0139,e_0140,e_0141,e_0142,e_0143,e_0144,e_0145,e_0146,e_0147,e_0148,e_0149,e_0150,e_0151,e_0152,e_0153,e_0154,e_0155,e_0156,e_0157,e_0158,e_0159,e_0160,e_0161,e_0162,e_0163,e_0164,e_0165,e_0166,e_0167,e_0168,e_0169,e_0170,e_0171,e_0172,e_0173,e_0174,e_0175,e_0176,e_0177,e_0178,e_0179,e_0180,e_0181,e_0182,e_0183,e_0184,e_0185,e_0186,e_0187,e_0188,e_0189,e_0190,e_0191,e_0192,e_0193,e_0194,e_0195,e_0196,e_0197,e_0198,e_0199,e_0200,e_0201,e_0202,e_0203,e_0204,e_0205,e_0206,e_0207,e_0208,e_0209,e_0210,e_0211,e_0212,e_0213,e_0214,e_0215,e_0216,e_0217,e_0218,e_0219,e_0220,e_0221,e_0222,e_0223,e_0224,e_0225,e_0226,e_0227,e_0228,e_0229,e_0230,e_0231,e_0232,e_0233,e_0234,e_0235,e_0236,e_0237,e_0238,e_0239,e_0240,e_0241,e_0242,e_0243,e_0244,e_0245,e_0246,e_0247,...,e_0774,e_0775,e_0776,e_0777,e_0778,e_0779,e_0780,e_0781,e_0782,e_0783,e_0784,e_0785,e_0786,e_0787,e_0788,e_0789,e_0790,e_0791,e_0792,e_0793,e_0794,e_0795,e_0796,e_0797,e_0798,e_0799,e_0800,e_0801,e_0802,e_0803,e_0804,e_0805,e_0806,e_0807,e_0808,e_0809,e_0810,e_0811,e_0812,e_0813,e_0814,e_0815,e_0816,e_0817,e_0818,e_0819,e_0820,e_0821,e_0822,e_0823,e_0824,e_0825,e_0826,e_0827,e_0828,e_0829,e_0830,e_0831,e_0832,e_0833,e_0834,e_0835,e_0836,e_0837,e_0838,e_0839,e_0840,e_0841,e_0842,e_0843,e_0844,e_0845,e_0846,e_0847,e_0848,e_0849,e_0850,e_0851,e_0852,e_0853,e_0854,e_0855,e_0856,e_0857,e_0858,e_0859,e_0860,e_0861,e_0862,e_0863,e_0864,e_0865,e_0866,e_0867,e_0868,e_0869,e_0870,e_0871,e_0872,e_0873,e_0874,e_0875,e_0876,e_0877,e_0878,e_0879,e_0880,e_0881,e_0882,e_0883,e_0884,e_0885,e_0886,e_0887,e_0888,e_0889,e_0890,e_0891,e_0892,e_0893,e_0894,e_0895,e_0896,e_0897,e_0898,e_0899,e_0900,e_0901,e_0902,e_0903,e_0904,e_0905,e_0906,e_0907,e_0908,e_0909,e_0910,e_0911,e_0912,e_0913,e_0914,e_0915,e_0916,e_0917,e_0918,e_0919,e_0920,e_0921,e_0922,e_0923,e_0924,e_0925,e_0926,e_0927,e_0928,e_0929,e_0930,e_0931,e_0932,e_0933,e_0934,e_0935,e_0936,e_0937,e_0938,e_0939,e_0940,e_0941,e_0942,e_0943,e_0944,e_0945,e_0946,e_0947,e_0948,e_0949,e_0950,e_0951,e_0952,e_0953,e_0954,e_0955,e_0956,e_0957,e_0958,e_0959,e_0960,e_0961,e_0962,e_0963,e_0964,e_0965,e_0966,e_0967,e_0968,e_0969,e_0970,e_0971,e_0972,e_0973,e_0974,e_0975,e_0976,e_0977,e_0978,e_0979,e_0980,e_0981,e_0982,e_0983,e_0984,e_0985,e_0986,e_0987,e_0988,e_0989,e_0990,e_0991,e_0992,e_0993,e_0994,e_0995,e_0996,e_0997,e_0998,e_0999,e_1000,e_1001,e_1002,e_1003,e_1004,e_1005,e_1006,e_1007,e_1008,e_1009,e_1010,e_1011,e_1012,e_1013,e_1014,e_1015,e_1016,e_1017,e_1018,e_1019,e_1020,e_1021,e_1022,e_1023
0,female,I loved this doctor!,-0.601180,0.63924,-1.060369,-0.493731,-0.560602,-1.008939,-0.598373,-0.672984,-0.640708,0.035111,-0.394858,1.125174,-0.809709,0.092503,-1.561161,-0.338891,-0.980971,-0.218150,-0.770218,0.518710,-0.154179,-0.465516,-0.636096,0.136777,-0.671058,0.887398,1.150699,-0.255780,-0.124599,-1.695018,-0.176872,-0.554525,0.420271,1.104314,-0.662254,-1.104489,-0.150347,-0.328107,-0.265293,-0.232560,-0.732200,0.102850,1.920283,0.345062,0.727855,-0.558262,-0.727879,0.068228,-

In [ ]:
show_doc(Autocoder.code_lda_topics)

---

### Autocoder.code_lda_topics

>      Autocoder.code_lda_topics (docs, df, k=10, n_features=10000)

*Encode texts as semantically meaningful vectors using Latent Dirichlet Alocation*

In [ ]:
comments = ["What is your favorite sitcom of all time?", 'I cannot wait to vote!']
df = pd.DataFrame({
    'over_18': ['yes', 'no'] * 5,
     'comments' : comments * 5,
      })
df.head()

,over_18,comments
0,yes,What is your favorite sitcom of all time?
1,no,I cannot wait to vote!
2,yes,What is your favorite sitcom of all time?
3,no,I cannot wait to vote!
4,yes,What is your favorite sitcom of all time?


In [ ]:
df = ac.code_lda_topics(df['comments'].values, df)

preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.
done.


In [ ]:
df.head()

,over_18,comments,time|favorite|sitcom,sitcom|vote|wait,wait|vote|favorite,time|sitcom|favorite,favorite|sitcom|wait,wait|time|favorite,sitcom|favorite|vote,vote|wait|time,favorite|vote|time,vote|favorite|sitcom
0,yes,What is your favorite sitcom of all time?,0.148763,0.093341,0.080723,0.128911,0.109816,0.084724,0.093611,0.080860,0.091758,0.087493
1,no,I cannot wait to vote!,0.085687,0.097749,0.142486,0.084145,0.086931,0.099608,0.091913,0.114741,0.093014,0.103728
2,yes,What is your favorite sitcom of all time?,0.148763,0.093341,0.080723,0.128911,0.109816,0.084724,0.093611,0.080860,0.091758,0.087493
3,no,I cannot wait to vote!,0.085687,0.097749,0.142486,0.084145,0.086931,0.099608,0.091913,0.114741,0.093014,0.103728
4,yes,What is your favorite sitcom of all time?,0.148763,0.093341,0.080723,0.128911,0.109816,0.084724,0.093611,0.080860,0.091758,0.087493


In [ ]:
show_doc(Autocoder.code_callable)

---

### Autocoder.code_callable

>      Autocoder.code_callable (docs, df, fn)

*Autocodes text for any user-specified function
The `fn` parameter must be a Callable and return a dictionary for each
text in `docs` where the keys are desired column names and values are scores
or probabilities.*

In [ ]:
reviews = ["I loved this doctor!", "This doctor was absolutely terrible."]
df = pd.DataFrame({
    'gender': ['female', 'male'],
     'review' : reviews,
      })
df.head()

,gender,review
0,female,I loved this doctor!
1,male,This doctor was absolutely terrible.


In [ ]:
def some_function(x):
    val = int('terrible' in x)
    return {'has_the_word_terrible?' : val}

In [ ]:
df = ac.code_callable(df.review.values, df, some_function)

In [ ]:
df.head()

,gender,review,has_the_word_terrible?
0,female,I loved this doctor!,0
1,male,This doctor was absolutely terrible.,1


In [ ]:
#| include: false
from nbdev import nbdev_export; nbdev_export()